# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/benfarrell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/benfarrell/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/benfarrell/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
import re
import sqlite3
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier

import pickle
import joblib

In [3]:
# connect to the database

database_filepath = 'DisasterReponse.db'

conn = sqlite3.connect(database_filepath)

title = database_filepath.split('.db')[0]

# load data from database
df = pd.read_sql(f'SELECT * FROM {title}', conn)

#Check dataframe
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#Assign data and target
#Get all columns except 'genre'
X = df['message']
y = df.iloc[:,4:]

In [10]:
type(y)

pandas.core.frame.DataFrame

### 2. Write a tokenization function to process your text data

In [11]:
#Stop Words
stops = stopwords.words('english')

In [12]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [13]:
def tokenize(text):
    
    #Replae urls
    
    text = re.sub(url_regex,'url',text)
    
    #Remove punctuation
    text = re.sub('[^a-zA-Z0-9 ]',' ',text)

    #instantiate word lemmatizer
    lemmatizer = WordNetLemmatizer()
   
    #tokenize text
    #Check for word in stops
    #Lemmatize
    #lower case and strip whitespace
    return [lemmatizer.lemmatize(tok).lower().strip() for tok in word_tokenize(text) if tok not in stops]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [31]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))   
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)

In [33]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [34]:
y_predict = pipeline.predict(X_test)

In [35]:
print(classification_report(y_test,y_predict,target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      2021
               request       0.91      0.44      0.59       461
                 offer       0.00      0.00      0.00         9
           aid_related       0.78      0.60      0.68      1082
          medical_help       0.67      0.06      0.10       213
      medical_products       0.50      0.03      0.06       125
     search_and_rescue       1.00      0.02      0.03        62
              security       0.00      0.00      0.00        42
              military       0.62      0.06      0.10        88
                 water       0.82      0.30      0.44       155
                  food       0.89      0.44      0.59       285
               shelter       0.85      0.27      0.41       215
              clothing       0.25      0.03      0.06        30
                 money       0.75      0.06      0.11        50
        missing_people       0.50      

/Users/benfarrell/anaconda3/envs/data1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/benfarrell/anaconda3/envs/data1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/benfarrell/anaconda3/envs/data1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

In [36]:
results = {}
target_names = list(y_test.columns)
for i in range(y_predict.shape[1]):
    results[target_names[i]] = classification_report(y_test.iloc[:,i],y_predict[:,i],output_dict=True)['macro avg']

precision = [results[k]['precision'] for k in results]
recall = [results[k]['recall'] for k in results]
f1_score = [results[k]['f1-score'] for k in results]

results_table = [('precision',precision),('recall',recall),('f1_score',f1_score)]

for r in results_table:
    print(f'Average {r[0]} of Model: {sum(r[1])/len(r[1])*100:.2f}%')

Average precision of Model: 74.22%
Average recall of Model: 57.72%
Average f1_score of Model: 59.39%


### 6. Improve your model
Use grid search to find better parameters. 

In [97]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=LogisticRegression(random_state=42)))],
 'verbose': False,
 'vect': CountVectorizer(),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=LogisticRegression(random_state=42)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': None,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'clf__estimator__C': 1.0,
 'clf__estimator__class_weight': None,
 'clf__estimator__d

In [106]:
parameters = {  'vect__max_df': (0.5,1.0),
                'tfidf__use_idf': (True, False)}

cv = GridSearchCV(pipeline,param_grid=parameters,verbose=1)

In [107]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/benfarrell/anaconda3/envs/data1/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/benfarrell/anaconda3/envs/data1/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the document

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=LogisticRegression(random_state=42)))]),
             param_grid={'tfidf__use_idf': (True, False),
                         'vect__max_df': (0.5, 1.0)},
             verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [108]:
y_predict2 = cv.predict(X_test)

In [109]:
results2 = {}
target_names = list(y_test.columns)
for i in range(y_predict.shape[1]):
    results2[target_names[i]] = classification_report(y_test.iloc[:,i],y_predict2[:,i],output_dict=True)['macro avg']

/Users/benfarrell/anaconda3/envs/data1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [110]:
precision = [results[k]['precision'] for k in results2]
recall = [results[k]['recall'] for k in results2]
f1_score = [results[k]['f1-score'] for k in results2]

results_table2 = [('precision',precision),('recall',recall),('f1_score',f1_score)]

for r in results_table2:
    print(f'Average {r[0]} of Model: {sum(r[1])/len(r[1])*100:.2f}%')

Average precision of Model: 78.90%
Average recall of Model: 58.04%
Average f1_score of Model: 59.70%


Model precision increased slightly, but recall and f1 score dropped slightly

### 8. Try improving your model further. Here are a few ideas:
* Attempt different Classifier
* Attempt to create custom feature

In [38]:
pipeline2 = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf',TfidfTransformer()),
                     ('clf',MultiOutputClassifier(AdaBoostClassifier()))
                    ])

In [39]:
pipeline2.fit(X_train,y_train)
y_predict = pipeline2.predict(X_test)

In [40]:
results = {}
target_names = list(y_test.columns)
for i in range(y_predict.shape[1]):
    results[target_names[i]] = classification_report(y_test.iloc[:,i],y_predict[:,i],output_dict=True)['macro avg']

precision = [results[k]['precision'] for k in results]
recall = [results[k]['recall'] for k in results]
f1_score = [results[k]['f1-score'] for k in results]

results_table = [('precision',precision),('recall',recall),('f1_score',f1_score)]

print('AdaBoostVlassifier')
for r in results_table:
    print(f'Average {r[0]} of Model: {sum(r[1])/len(r[1])*100:.2f}%')

/Users/benfarrell/anaconda3/envs/data1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AdaBoostVlassifier
Average precision of Model: 75.15%
Average recall of Model: 65.43%
Average f1_score of Model: 68.23%


### 9. Export your model as a pickle file

In [51]:
import os

In [53]:
pkl_file = os.getcwd() + '/model2_pickle.pkl'

In [54]:
with open(pkl_file,'wb') as file:
    pickle.dump(pipeline2, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.